In [1]:
print(1)


1


In [ ]:
import ir_datasets
import email
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import gc
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from config import *
import re
import numpy as np

### Data set-up, pre-processing and model set-up, evaluation metrics, experiment set-up

In [3]:
import sys
sys.path.append("../scripts/")

In [4]:
from dataset import load_sara
from eval import jupyter_evaluation
from model import llm_experiment, post_process_split_docs
from models import get_model_version
from preprocess_sara import proccutit


#s = load_sara()
#proc = proccutit(s)
#tokenizer, model = get_model_version('get_mistral', "mistralai/Mistral-7B-Instruct-v0.2", 'main', 'auto')
#jupyter_evaluation(labels, preds)
#llm_experiment(dataset, prompt_strategy, model, tokenizer, end_prompt=None):
#post_process_split_docs(mr, fpr, pre, df)

In [10]:
tokenizer, model = get_model_version('get_mistral', "mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.58s/it]


In [5]:
def get_sample_n(data, n):
    return data.sample(n=n, random_state=1)

def get_sample_frac(data, frac):
    return data.sample(frac=frac, random_state=1)

### Main experiment - main call

In [34]:
# Get dataset
# Preprocess
# Get model
# Get prompt
# Run evaluation
# Display evaluation metrics

def main_experiment(portion, portion_value, proc_meth, prompt_strategy, end_prompt, tokenizer, model):
    sara_df = load_sara()
    if portion == 'n':
        samp = get_sample_n(sara_df, portion_value)
    else:
        samp = get_sample_frac(sara_df, portion_value)

    if proc_meth:
        proc_samp = proccutit(samp)
    else:
        proc_samp = samp
        
    #tokenizer, model = get_model_version(m, v)
    preds, truths, model_responses, further_processing_required = llm_experiment(proc_samp, prompt_strategy, model, tokenizer, end_prompt)
    new_preds, new_truths = post_process_split_docs(model_responses, further_processing_required, preds, sara_df)
    jupyter_evaluation(new_truths, new_preds)
    return model_responses

### Investigation

In [29]:
def post_process_classification(classification):
    match_string = classification.lower()
    if "non-personal" in match_string:
        return 0
    elif "personal" in match_string:
        return 1
    else:
        return None

In [32]:
def b1(document):
  return f"[INST] Your task is to determine if the email message from a work email contains personal information. Purely personal messages include personal information and do not include any relation to work being done. Personal but in a professional context messages include personal information that are related to work, for example comments about the quality of people's work and expressions of feelings about employee treatment. Does the message contain purely personal information or information that is personal a professional context? \nMessage: {document} \n[/INST] \n The text does"

def b1(document):
  return f"[INST] Your task is to determine if the email message from a work email contains personal information. Purely personal messages include personal information and do not include any relation to work being done. Personal but in a professional context messages include personal information that are related to work, for example comments about the quality of people's work and expressions of feelings about employee treatment. Does the message contain purely personal information or information that is personal a professional context? If the message does contain personal information respond with 'personal', otherwise respond with 'non-personal'. \nMessage: {document} \n[/INST] \nClassification: "


In [1]:
#mr = main_experiment('n', 10, True, b1, '\n[/INST]', tokenizer, model)
